In [1]:
import pandas as pd

# Import bibliothèque de manipulation de dataframe
import pandas as pd
import numpy as np

# Import des bibliothèques de viz
import matplotlib.pyplot as plt
import seaborn as sns

# Import split data
from sklearn.model_selection import train_test_split

# Import modèles de ML Supervisé Régression
from sklearn.linear_model import LinearRegression

# Import modèles de ML Supervisé Classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

# Import modèle de ML NON Supervisé
from sklearn.neighbors import NearestNeighbors

# Import des métriques
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# Import outil standardisation de la donnée
from sklearn.preprocessing import StandardScaler

# Import pipeline
from sklearn.pipeline import Pipeline

# Gestion des warnings
import warnings

In [2]:
df_name_basics = pd.read_csv("data/original/name.basics.tsv.gz", sep = '\t', na_values='\\N')
dfTITLE = pd.read_csv('data/original/title.principals.tsv.gz',sep='\t',na_values="\\N")
dfTITLE = dfTITLE.drop(columns=['ordering', 'job', 'characters'])
dfTITLE = dfTITLE.rename(columns={'tconst': 'ID_film'})
dfTITLE = dfTITLE.rename(columns={'category': 'Rôle'})
dfTITLE = dfTITLE[dfTITLE["Rôle"].isin(["actor", "actress", "director", "producer"])]
dfTITLE2 = dfTITLE.merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
dfTITLE2 = dfTITLE2.drop_duplicates()
dfTITLE2 = dfTITLE2.dropna()
dfTITLE2.head(10)

,ID_film,nconst,Rôle,primaryName
0,tt0000001,nm0005690,director,William K.L. Dickson
1,tt0000001,nm0005690,producer,William K.L. Dickson
2,tt0000002,nm0721526,director,Émile Reynaud
3,tt0000003,nm0721526,director,Émile Reynaud
4,tt0000003,nm1770680,producer,Julien Pappé
5,tt0000003,nm0721526,producer,Émile Reynaud
6,tt0000004,nm0721526,director,Émile Reynaud
7,tt0000005,nm0443482,actor,Charles Kayser
8,tt0000005,nm0653042,actor,John Ott
9,tt0000005,nm0249379,producer,Thomas A. Edison


In [3]:
dfTMDB = pd.read_csv('data/original/tmdb_full.clean.csv.gz',sep=',',na_values="\\N") #fichier clean seb
dfTMDB = dfTMDB.rename(columns={"imdb_id": "ID_film"})
dfTMDB["vote_average"] = dfTMDB["vote_average"].astype(str) # Passage en str pour l'impliquer dans le groupby
dfTMDB["runtime"] = dfTMDB["runtime"].astype(str) # Passage en str pour l'impliquer dans le groupby
dfTMDB = dfTMDB.groupby("ID_film")[["genres","vote_average","runtime"]].agg(",".join).reset_index() # regroupe les lignes ID film par genres
dfTMDB["vote_average"] = round(dfTMDB["vote_average"].astype(float),1) # repassage en float et j'affiche qu'un chiffre après la virgule
dfTMDB["runtime"] = dfTMDB["runtime"].astype(int) # repassage en int
dfTMDB["genres"] = dfTMDB["genres"].str.replace("[\[\]']", "", regex=True) # supprime les [ pour l'affichage
dfTMDB.head(10)

,ID_film,genres,vote_average,runtime
0,tt0000001,Documentary,5.2,1
1,tt0000002,Animation,5.3,10
2,tt0000003,"Comedy, Animation",6.0,4
3,tt0000004,Animation,4.4,15
4,tt0000005,"Drama, Documentary",5.6,1
5,tt0000006,,3.2,1
6,tt0000007,Documentary,5.0,1
7,tt0000008,Documentary,4.9,1
8,tt0000010,Documentary,6.7,1
9,tt0000011,Documentary,4.5,1


In [4]:
dfTMDBfull = pd.read_csv('data/original/tmdb_full (1).csv.gz',sep=',',na_values="\\N") #fichier brut sans modif
dfTMDBfull = dfTMDBfull[dfTMDBfull['production_countries'].str.contains("US|FR", na=False)] # ne prend que les lignes qui contiennent US et FR
dfTMDBfull.rename(columns={"imdb_id": "ID_film"}, inplace=True)
dfTMDBfull = dfTMDBfull[['production_countries', 'ID_film','poster_path','vote_average','runtime']]
dfTMDBfull.head(10)

/var/folders/r8/68mtf6pj63jfp7zq8kdd3bfm0000gn/T/ipykernel_60991/845861435.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  dfTMDBfull = pd.read_csv('data/original/tmdb_full (1).csv.gz',sep=',',na_values="\\N") #fichier brut sans modif


,production_countries,ID_film,poster_path,vote_average,runtime
0,['US'],tt0029927,/zBiHKhXklvTFwj4M1uEUcQGAVJ.jpg,7.214,70
2,"['DE', 'FR', 'IT', 'JP', 'PL']",tt0055747,/aup2QCYCsyEeQfpboXy0f4uj8aE.jpg,6.700,110
5,['US'],tt0113101,/75aHn1NOYXh4M7L5shoeQ6NGykP.jpg,5.758,98
6,['US'],tt0107286,/3rvvpS9YPM5HB2f4HYiNiJVtdam.jpg,6.600,109
9,['US'],tt0076759,/6FfCtAuVAW8XJjZ7eWeLibRLWTw.jpg,8.208,121
10,['US'],tt0266543,/5lc6nQc0VhWFYFbNv016xze8Jvy.jpg,7.823,100
11,['US'],tt0109830,/arw2vcBveWOVZr6pxd9XTd1TdQa.jpg,8.481,142
12,['US'],tt0169547,/wby9315QzVKdW9BonAefg8jGTTb.jpg,8.027,122
13,['US'],tt0033467,/sav0jxhqiH0bPr2vZFU0Kjt2nZL.jpg,8.021,119
14,"['AR', 'DK', 'FI', 'FR', 'DE', 'IS', 'IT', 'NL...",tt0168629,/8Wdd3fQfbbQeoSfWpHrDfaFNhBU.jpg,7.881,141


In [5]:
df = pd.read_csv('data/original/title.basics.tsv.gz',sep='\t',na_values="\\N")

df.drop(columns=["primaryTitle", "runtimeMinutes", "endYear"], inplace=True)

df.rename(columns={
    "tconst": "ID_film",
    "titleType": "Type",
    "originalTitle": "Titre",
    "startYear": "Année",
    "genres": "Genres (max 3)"
}, inplace=True)

df = df[df["Année"].notna()]
df["Année"] = df["Année"].astype(int)
df["Genres (max 3)"] = df["Genres (max 3)"].fillna("inconnu")
df = df[df["isAdult"] == 0] #Garde les lignes ou adulte = 0

df_final2 = pd.merge(df, dfTITLE2, on="ID_film", how="left") # jointure entre title principals et df_final pour les prénoms noms/role des gens
df_final2 = df_final2[df_final2["Type"] == "movie"] # ne garder que les films

df_final2.rename(columns={
    "Titre_x": "Titre",
    "primaryName": "Prénom",
    "primaryProfession": "Rôle"
}, inplace=True)

df_final3 = pd.merge(df_final2, dfTMDB, on="ID_film", how="left") # jointure entre df_final2 et dfTMDB pour les genres de film

df_final3["Genres"] = df_final3['Genres (max 3)'] + "," + df_final3['genres'] # je rejoins les colonnes "genre" entre eux en une chaine de caractère separé par virgule
df_final3["Genres"] = df_final3["Genres"].fillna(df_final3["Genres (max 3)"]) #je remplis les "genres" manquant par la valeur 'genres max 3' de la table title.basics
df_final3 = df_final3.drop(columns=['genres', 'Genres (max 3)', 'vote_average', 'runtime'])

df_final4 = pd.merge(df_final3, dfTMDBfull, on="ID_film", how="left")
df_final4 = df_final4.dropna(subset=['production_countries']) # drop les erreurs de production countries
df_final4 = df_final4.drop(columns='isAdult') # supprimée car par nécessaire

# suppr des colonnes NaN
df_final5 = df_final4
df_final5 = df_final5.drop_duplicates(subset=["ID_film","Genres"], keep="first") # supprime les lignes du df si les valeurs ID_film et Genres sont identiques

df_final5['Genres'] = (
    df_final5['Genres']
    .str.replace(" ", "")        # enlève les espaces
    .str.split(',')              # transforme en liste
    .apply(lambda x: sorted(set(x)))  # enlève doublons
    .str.join(',')               # remet en chaîne)
)

# passage en STR pour pas l'appliquer dans le ML
df_final5['runtime'] = df_final5['runtime'].astype(str)
df_final5['vote_average'] = df_final5['vote_average'].astype(str)
df_final5['Année'] = df_final5['Année'].astype(str)

# On supprime les lignes où AU MOINS UNE colonne vaut 'Inconnu'
df_final5 = df_final5[~df_final5.isin(["inconnu", "Inconnu"]).any(axis=1)]

df_final5.head(10)

/var/folders/r8/68mtf6pj63jfp7zq8kdd3bfm0000gn/T/ipykernel_60991/769777911.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/original/title.basics.tsv.gz',sep='\t',na_values="\\N")
/var/folders/r8/68mtf6pj63jfp7zq8kdd3bfm0000gn/T/ipykernel_60991/769777911.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final5['Genres'] = (
/var/folders/r8/68mtf6pj63jfp7zq8kdd3bfm0000gn/T/ipykernel_60991/769777911.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,ID_film,Type,Titre,Année,nconst,Rôle,Prénom,Genres,production_countries,poster_path,vote_average,runtime
5,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,nm0714557,director,Enoch J. Rector,"Documentary,News,Sport",['US'],/nL7aYlGYJZU7Gx8kFu9bTp3QYRu.jpg,5.526,19.0
33,tt0000591,movie,L'enfant prodigue,1907,nm0906197,actor,Georges Wague,Drama,['FR'],NaN,0.0,90.0
120,tt0001101,movie,Abraham Lincoln's Clemency,1910,nm0923594,actor,Leopold Wharton,",inconnu",['US'],NaN,0.0,0.0
223,tt0001422,movie,Trail to the West,1910,nm0001908,actor,Gilbert M. 'Broncho Billy' Anderson,"Western,inconnu",['US'],NaN,0.0,0.0
371,tt0001790,movie,Les misérables - Époque 1: Jean Valjean,1913,nm0470307,actor,Henry Krauss,"Drama,History",['FR'],NaN,1.0,60.0
457,tt0002031,movie,As You Like It,1912,nm0007221,actor,Maurice Costello,"Comedy,inconnu",['US'],/O8oslUT10jwIyxBGVTvagNjUPS.jpg,1.0,33.0
483,tt0002101,movie,Cleopatra,1912,nm0306947,actress,Helen Gardner,"Drama,History",['US'],/veURP5WEMB7mKLkCbqYHkJm0O0g.jpg,3.219,88.0
551,tt0002199,movie,From the Manger to the Cross,1912,nm0087381,actor,R. Henderson Bland,"Biography,Drama",['US'],/wsRp6nRPam7F7ShyhRtswBRE283.jpg,5.325,71.0
625,tt0002375,movie,La mort du duc d'Enghien,1912,nm0135053,actor,Paul Capellani,"Drama,History,inconnu",['FR'],/1D9MCUxnl4nzFVlJNsppP6Sdk94.jpg,0.0,15.0
635,tt0002405,movie,Oliver Twist,1912,nm0329522,actor,Nat C. Goodwin,Drama,['US'],/j6FbBm1PVkPd4XmNzHAzRQ90frl.jpg,0.0,0.0


In [6]:
df_final4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 739899 entries, 5 to 5002001
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   ID_film               739899 non-null  object 
 1   Type                  739899 non-null  object 
 2   Titre                 739899 non-null  object 
 3   Année                 739899 non-null  int64  
 4   nconst                739789 non-null  object 
 5   Rôle                  739789 non-null  object 
 6   Prénom                739789 non-null  object 
 7   Genres                739899 non-null  object 
 8   production_countries  739899 non-null  object 
 9   poster_path           705323 non-null  object 
 10  vote_average          739899 non-null  float64
 11  runtime               739899 non-null  float64
dtypes: float64(2), int64(1), object(9)
memory usage: 73.4+ MB


In [7]:
matrix = df_final5[df_final5["Titre"].str.contains("rocky", case=False, na=False)]
matrix

,ID_film,Type,Titre,Année,nconst,Rôle,Prénom,Genres,production_countries,poster_path,vote_average,runtime
156261,tt0025729,movie,Rocky Rhodes,1934,nm0427659,actor,Buck Jones,"Drama,Western",['US'],/x2wuZeV89hR7T0Epww4YeC3qABJ.jpg,0.0,64.0
157899,tt0025939,movie,Rocky Mountain Mystery,1935,nm0000068,actor,Randolph Scott,"Drama,Mystery,Western",['US'],/J9o1zwnb3WaTUfGOCBh0n68dk.jpg,5.5,63.0
220882,tt0032996,movie,Rocky Mountain Rangers,1940,nm0515292,actor,Robert Livingston,"Drama,Western",['US'],/av4ayunv4X3AjCRoGYS0XUmtlN7.jpg,0.0,58.0
290738,tt0040743,movie,Rocky,1948,nm0001522,actor,Roddy McDowall,"Drama,Western",['US'],/iPUZlfz01BHn6lzZ6tUUJonyjKV.jpg,0.0,76.0
309563,tt0042899,movie,Rocky Mountain,1950,nm0001224,actor,Errol Flynn,"Action,Adventure,Drama,Western",['US'],/lU6f8rOwmnNu9N4wOBkmS6D0kR.jpg,5.7,83.0
570574,tt0073629,movie,The Rocky Horror Picture Show,1975,nm0000347,actor,Tim Curry,"Comedy,Fantasy,Horror,Musical,ScienceFiction","['GB', 'US']",/3pyE6ZqDbuJi7zrNzzQzcKTWdmN.jpg,7.494,100.0
581665,tt0075148,movie,Rocky,1976,nm0000230,actor,Sylvester Stallone,"Drama,Romance,Sport",['US'],/hEjK9A9BkNXejFW4tfacVAEHtkn.jpg,7.786,119.0
614866,tt0079817,movie,Rocky II,1979,nm0000230,actor,Sylvester Stallone,"Drama,Sport",['US'],/t2WuqQ0Jbyaql1WXNFx9dr6Ldpt.jpg,7.254,119.0
648683,tt0084602,movie,Rocky III,1982,nm0000230,actor,Sylvester Stallone,"Drama,Sport",['US'],/3avfzaUDxMieyEgGPopVvh8ABXR.jpg,6.9,99.0
685539,tt0089927,movie,Rocky IV,1985,nm0000230,actor,Sylvester Stallone,"Drama,Sport",['US'],/jmvpwgW5M2kduR9zB0q8qGFC4zM.jpg,7.1,91.0


In [9]:
test = df_final5['Prénom'].value_counts() # on compte cmb de fois chaqué prénom apparaît
test2 = test[test > 2].index # je récupère les prénoms qui apparaissent plus de deux fois
df_final5["plusde2"] = df_final5["Prénom"].where(df_final5["Prénom"].isin(test2)) # nouvelle colonne -> Si apparait plus de deux fois garde dans la colonne
X_prenom = df_final5['plusde2'].str.get_dummies() #encodage one-hot des prénoms

X_role = df_final5["Rôle"].str.get_dummies(sep=",") # encodage one-hot des rôles

X_genre = df_final5["Genres"].str.get_dummies(sep=",") # encodage one-hot des genres
df_concat = pd.concat([df_final5, X_genre, X_prenom, X_role], axis=1) # on fusionne le df final avec les colonnes encodées
df_num = df_concat.select_dtypes(include='number') # on garde que les colonnes numériques
df_num.head(10)

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Zoë Bell,Zsa Zsa Gabor,Ève Francis,Élodie Bouchez,Émilie Dequenne,Éric Caravaca,actor,actress,director,producer
5,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
33,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
120,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
223,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
371,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
457,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
483,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
551,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
625,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
635,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0


In [10]:
df_final5

,ID_film,Type,Titre,Année,nconst,Rôle,Prénom,Genres,production_countries,poster_path,vote_average,runtime,plusde2
5,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,nm0714557,director,Enoch J. Rector,"Documentary,News,Sport",['US'],/nL7aYlGYJZU7Gx8kFu9bTp3QYRu.jpg,5.526,19.0,NaN
33,tt0000591,movie,L'enfant prodigue,1907,nm0906197,actor,Georges Wague,Drama,['FR'],NaN,0.0,90.0,NaN
120,tt0001101,movie,Abraham Lincoln's Clemency,1910,nm0923594,actor,Leopold Wharton,",inconnu",['US'],NaN,0.0,0.0,NaN
223,tt0001422,movie,Trail to the West,1910,nm0001908,actor,Gilbert M. 'Broncho Billy' Anderson,"Western,inconnu",['US'],NaN,0.0,0.0,Gilbert M. 'Broncho Billy' Anderson
371,tt0001790,movie,Les misérables - Époque 1: Jean Valjean,1913,nm0470307,actor,Henry Krauss,"Drama,History",['FR'],NaN,1.0,60.0,Henry Krauss
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5001268,tt9907782,movie,Eight for Silver,2021,nm2933542,actor,Boyd Holbrook,"Fantasy,Horror,Mystery",['US'],/bewmBcjJxHeipS1PNu9Lfyvt3zY.jpg,6.9,111.0,Boyd Holbrook
5001282,tt9908390,movie,Le lion,2020,nm0200702,actor,Dany Boon,Comedy,['FR'],/zfLuACsPcwH6aonTKukn96d1qpp.jpg,5.328,95.0,Dany Boon
5001707,tt9913056,movie,Swarm Season,2019,nm1502645,director,Sarah Christman,"Documentary,Drama",['US'],/s1DBGHegIs6A41Co9vlfLEAhweA.jpg,2.0,85.0,NaN
5001865,tt9914644,movie,9/11: Escape from the Towers,2018,nm2603587,director,Grace Chapman,"Documentary,History",['US'],/t7W1tt7fhs78Htp0XME4WNVDOM3.jpg,7.0,86.0,NaN


In [11]:
scaler_knn = StandardScaler()
X_knn_scaled = scaler_knn.fit_transform(df_num)

nn_model = NearestNeighbors(algorithm='auto', metric='cosine')
nn_model.fit(X_knn_scaled)

,n_neighbors,5
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'cosine'
,p,2
,metric_params,None
,n_jobs,None


In [25]:
def find_neighbors_by_name(
    film_name: str,
    k: int,
    nn_model: NearestNeighbors,
    scaler: StandardScaler,
    features_df: pd.DataFrame,
    original_df: pd.DataFrame
) -> None:

    # trouver l’index à partir du titre
    if film_name not in features_df.index:
        matches = original_df.index[original_df["Titre"] == film_name]

        if len(matches) == 0:
            print(f"Erreur : le film '{film_name}' n'existe pas dans les données utilisées.")
            return

        film_name = matches[0]  # on remplace le titre par l’index

    # Récupérer les features du film cible
    film_features = features_df.loc[[film_name]]  # Garder le format DataFrame
    film_features_scaled = scaler.transform(film_features)

    distances, indices = nn_model.kneighbors(film_features_scaled, n_neighbors=k)

    neighbor_indices_in_features_df = indices[0]
    neighbor_distances = distances[0]

    # Indices valides dans features_df
    valid_indices = [idx for idx in neighbor_indices_in_features_df if idx < len(features_df)]
    if not valid_indices:
        print("Aucun voisin trouvé")
        return

    print(f"\n--- Plus proches film voisins '{film_name}' ---")
    print("Film cible:")
    print(original_df.loc[[film_name]])

    neighbor_info = original_df.iloc[valid_indices].copy()

    # distances alignées : une distance par ligne
    neighbor_info["distance"] = neighbor_distances[:len(neighbor_info)]

    # Retirer le film lui-même
    neighbor_info = neighbor_info[neighbor_info.index != film_name]

    # Dédoublonner par film (on garde la ligne la plus proche pour chaque ID_film)
    neighbor_info = neighbor_info.sort_values("distance").drop_duplicates(subset=["ID_film"], keep="first")

    # Garder seulement k films uniques
    neighbor_info = neighbor_info.nsmallest(k, "distance")

    print(f"\nFilms voisins trouvés :")
    print(neighbor_info[[
        "ID_film"
    ]])

In [ ]:
# APPEL: 
find_neighbors_by_name(
    film_name="Rocky",
    k=3,
    nn_model=nn_model,
    scaler=scaler_knn,
    features_df=df_num,
    original_df=df_concat
)


--- Plus proches film voisins '290738' ---
Film cible:
          ID_film   Type  Titre Année     nconst   Rôle          Prénom  \
290738  tt0040743  movie  Rocky  1948  nm0001522  actor  Roddy McDowall   

               Genres production_countries                       poster_path  \
290738  Drama,Western               ['US']  /iPUZlfz01BHn6lzZ6tUUJonyjKV.jpg   

        ... Zoë Bell Zsa Zsa Gabor Ève Francis  Élodie Bouchez  \
290738  ...        0             0           0               0   

        Émilie Dequenne  Éric Caravaca  actor  actress  director  producer  
290738                0              0      1        0         0         0  

[1 rows x 4453 columns]

Films voisins trouvés :
          ID_film
330437  tt0045188
475778  tt0061514
469110  tt0060636
307066  tt0042642


In [ ]:
# -------------------------------------------------------------
# export
# -------------------------------------------------------------

import joblib
import pandas as pd

# df --> parquet
df_concat.to_parquet("df_concat.parquet")

# les colonnes de df_num pour le re-créer
feature_columns = df_num.columns.tolist()
pd.Series(feature_columns, name="feature").to_csv("feature_columns.csv", index=False)

# modèle
joblib.dump(scaler_knn, "scaler.joblib")
joblib.dump(nn_model, "nn_model.joblib")
